## Cleaning

In [284]:
import pandas as pd
import numpy as np
import math
import re
from datetime import datetime
import time

In [147]:
df = pd.read_csv('House_Votes.csv', encoding = 'unicode_escape')

In [148]:
df['dnum'] = df['district'].apply(lambda x: '{0:0>2}'.format(x))

In [149]:
df['dnum'] = df.dnum.apply(str)

In [150]:
df['year'] = df.year.apply(str)

In [151]:
df['district_id'] = df[['state_po', 'dnum']].apply(lambda x: '-'.join(x), axis=1)

In [152]:
df['yr_district_id'] = df[['year', 'district_id']].apply(lambda x: '-'.join(x), axis=1)

In [153]:
df.drop(['state_po', 'state_fips', 'state_cen', 'state_ic',
       'office', 'district', 'mode', 'unofficial', 'version', 'dnum'], axis = 1, inplace = True)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29636 entries, 0 to 29635
Data columns (total 12 columns):
year              29636 non-null object
state             29636 non-null object
stage             29606 non-null object
runoff            20980 non-null object
special           29636 non-null bool
candidate         27515 non-null object
party             26208 non-null object
writein           29636 non-null bool
candidatevotes    29636 non-null int64
totalvotes        29636 non-null int64
district_id       29636 non-null object
yr_district_id    29636 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 2.3+ MB


In [155]:
df = df[(df.stage != 'pri') & (df.runoff != True) & (df.special != True)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29538 entries, 0 to 29635
Data columns (total 12 columns):
year              29538 non-null object
state             29538 non-null object
stage             29538 non-null object
runoff            20912 non-null object
special           29538 non-null bool
candidate         27422 non-null object
party             26115 non-null object
writein           29538 non-null bool
candidatevotes    29538 non-null int64
totalvotes        29538 non-null int64
district_id       29538 non-null object
yr_district_id    29538 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 2.5+ MB


In [156]:
df.year.value_counts()

2000    1608
1996    1594
1992    1524
2010    1500
2016    1440
2002    1431
2012    1427
2008    1381
1998    1373
2018    1359
2014    1357
2006    1344
2004    1340
1976    1306
1994    1301
1980    1285
1982    1275
1984    1171
1990    1155
1978    1130
1988    1129
1986    1108
Name: year, dtype: int64

In [142]:
22*435

9570

In [36]:
df.party.unique()

array(['democrat', 'republican', nan, 'prohibition', 'national democrat',
       'independent', 'libertarian', 'peace and freedom',
       'american independent', 'u.s. labor', 'socialist workers',
       'american', 'la raza unida', 'indpendent', 'communist',
       'conservative', 'socialist labor',
       'independents for godly government', "people's", 'workers',
       'white power', 'human rights', 'independent american',
       'new majority', 'labor', 'regular democracy', 'pro-life',
       'restoration', 'individual needs center',
       "independent taxpayer's watchdog", 'politicians are crooks',
       'jobs, equality, peace', 'consumer action', 'bring us together',
       'individual americans independence', 'silent majority',
       "people's independent", 'liberal', 'mayflower', 'coequal citizens',
       'revolutionary workers', 'independent conservatives',
       'constitution', 'citizens for haas', 'united states labor',
       'aloha democratic', 'socialist', "worker'

In [64]:
df.party.value_counts()


democrat                                      9060
republican                                    8839
libertarian                                   2564
independent                                   1127
conservative                                   621
green                                          491
natural law                                    371
liberal                                        266
working families                               245
independence                                   229
right to life                                  225
reform                                         173
peace and freedom                              165
constitution                                   140
socialist workers                              132
american independent                           104
democratic-farmer-labor                        101
american                                        72
none                                            67
u.s. taxpayers                 

In [157]:
df.head()

,year,state,stage,runoff,special,candidate,party,writein,candidatevotes,totalvotes,district_id,yr_district_id
0,1976,Alabama,gen,False,False,Bill Davenport,democrat,False,58906,157170,AL-01,1976-AL-01
1,1976,Alabama,gen,False,False,Jack Edwards,republican,False,98257,157170,AL-01,1976-AL-01
2,1976,Alabama,gen,False,False,NaN,NaN,True,7,157170,AL-01,1976-AL-01
3,1976,Alabama,gen,False,False,J. Carole Keahey,democrat,False,66288,156362,AL-02,1976-AL-02
4,1976,Alabama,gen,False,False,NaN,NaN,True,5,156362,AL-02,1976-AL-02


In [93]:
df['year'] = df.year.apply(int)

In [158]:
def func(x):
     return pd.Series(dict(candidatevotes = x['candidatevotes'].max(), 
                           totalvotes = x['candidatevotes'].sum()))

In [159]:
winner = pd.DataFrame(df.groupby(['yr_district_id']).apply(func))

In [160]:
winner.reset_index(inplace=True)

In [161]:
winner

,yr_district_id,candidatevotes,totalvotes
0,1976-AK-00,83722,118208
1,1976-AL-01,98257,157170
2,1976-AL-02,90069,156362
3,1976-AL-03,106935,108048
4,1976-AL-04,141490,176022
5,1976-AL-05,113553,113560
6,1976-AL-06,92113,162518
7,1976-AL-07,110496,110501
8,1976-AR-01,116217,168782
9,1976-AR-02,144780,167607


In [188]:
winner_df = pd.merge(winner, df[['year', 'state', 'candidate', 'party', 'writein',
       'candidatevotes', 'district_id', 'yr_district_id']], on=['yr_district_id', 'candidatevotes'])
winner_df

,yr_district_id,candidatevotes,totalvotes,year,state,candidate,party,writein,district_id
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02


In [189]:
winner_df['voteshare'] = winner_df.candidatevotes / winner_df.totalvotes

In [190]:
winner_df.voteshare.describe()

count    9552.000000
mean        0.670675
std         0.134570
min         0.274997
25%         0.573844
50%         0.648718
75%         0.731232
max         1.000000
Name: voteshare, dtype: float64

In [191]:
len(winner_df[winner_df.voteshare < 0.5])

391

In [192]:
winner_df.loc[winner_df['voteshare'] < 0.275]


,yr_district_id,candidatevotes,totalvotes,year,state,candidate,party,writein,district_id,voteshare
8858,2016-LA-03,77671,282443,2016,Louisiana,Clay Higgins,republican,False,LA-03,0.274997


In [193]:
winner_df.columns

Index(['yr_district_id', 'candidatevotes', 'totalvotes', 'year', 'state',
       'candidate', 'party', 'writein', 'district_id', 'voteshare'],
      dtype='object')

In [219]:
winner_df.columns = ['yr_district_id', 'winner_votes', 'totalvotes', 'year', 'state',
       'winner', 'winner_party', 'writein', 'district_id', 'winner_voteshare']

In [220]:
winner_df.head()

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820


In [197]:
winner_df.loc[winner_df["winner_party"] == 'independent']

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,voteshare
3457,1990-VT-00,117522,209856,1990,Vermont,Bernie Sanders,independent,False,VT-00,0.560013
3892,1992-VT-00,162724,281162,1992,Vermont,Bernard Sanders,independent,False,VT-00,0.578755
4327,1994-VT-00,105502,211449,1994,Vermont,Bernard Sanders,independent,False,VT-00,0.498948
4574,1996-MO-08,112472,222854,1996,Missouri,Jo Ann Emerson,independent,False,MO-08,0.504689
4749,1996-VT-00,140678,254706,1996,Vermont,Bernard Sanders,independent,False,VT-00,0.552315
5184,1998-VT-00,136403,215133,1998,Vermont,Bernard Sanders,independent,False,VT-00,0.634040
5612,2000-VA-05,143312,212705,2000,Virginia,"Virgil H. Goode, Jr.",independent,False,VA-05,0.673759
5619,2000-VT-00,196118,283366,2000,Vermont,Bernard Sanders,independent,False,VT-00,0.692101
6055,2002-VT-00,144880,225255,2002,Vermont,Bernard Sanders,independent,False,VT-00,0.643182
6490,2004-VT-00,205774,305008,2004,Vermont,Bernard Sanders,independent,False,VT-00,0.674651


In [198]:
winner_df["winner_party"]= winner_df["winner_party"].replace("democratic-farmer-labor", "democrat") 

In [199]:
winner_df["winner_party"]= winner_df["winner_party"].replace("foglietta (democrat)", "democrat") 

In [200]:
winner_df["winner_party"]= winner_df["winner_party"].replace("independent-republican", "republican") 

In [201]:
winner_df.loc[winner_df["winner"] == 'Bernie Sanders', 'winner_party'] = 'democrat'
winner_df.loc[winner_df["winner"] == 'Bernard Sanders', 'winner_party'] = 'democrat'
winner_df.loc[winner_df["winner"] == 'Jo Ann Emerson', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Virgil H. Goode, Jr.', 'winner_party'] = 'republican'

In [217]:
winner_df.winner_party.value_counts()

democrat      5144
republican    4405
Name: winner_party, dtype: int64

In [239]:
winner_df[winner_df.isna().any(axis=1)]

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare


In [234]:
winner_df.loc[winner_df["winner"] == 'Thomas G. Tancredo', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Mo Brooks', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Christopher H. Smith', 'winner_party'] = 'republican'
winner_df.loc[winner_df["winner"] == 'Mike Gallagher', 'winner_party'] = 'republican'

In [238]:
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'winner_votes'] = 1
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'totalvotes'] = 1
winner_df.loc[winner_df["winner"] == 'Frederica S. Wilson', 'winner_voteshare'] = 1

In [206]:
df1.party.value_counts()

democrat                                      9026
republican                                    8801
libertarian                                   2559
independent                                   1115
conservative                                   620
green                                          491
natural law                                    371
liberal                                        266
working families                               245
independence                                   229
right to life                                  225
reform                                         173
peace and freedom                              165
constitution                                   140
socialist workers                              131
american independent                           104
democratic-farmer-labor                        101
american                                        72
none                                            67
no party affiliation           

In [205]:
df1 = df.copy()

In [207]:
df1["party"]= df1["party"].replace("green", "democrat") 

In [208]:
df1["party"]= df1["party"].replace("liberal", "democrat") 

In [209]:
df1["party"]= df1["party"].replace("democratic-farmer-labor", "democrat") 

In [210]:
df1["party"]= df1["party"].replace("socialist workers", "democrat") 

In [211]:
df1["party"]= df1["party"].replace("foglietta (democrat)", "democrat") 

In [212]:
df1["party"]= df1["party"].replace("independent-republican", "republican") 

In [213]:
df1["party"]= df1["party"].replace("libertarian", "republican") 

In [214]:
df1["party"]= df1["party"].replace("conservative", "republican") 

In [215]:
df1.loc[df1["candidate"] == 'Bernie Sanders', 'party'] = 'democrat'
df1.loc[df1["candidate"] == 'Bernard Sanders', 'party'] = 'democrat'
df1.loc[df1["candidate"] == 'Jo Ann Emerson', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Virgil H. Goode, Jr.', 'party'] = 'republican'

In [250]:
df1.loc[df1["candidate"] == 'Thomas G. Tancredo', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Mo Brooks', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Christopher H. Smith', 'party'] = 'republican'
df1.loc[df1["candidate"] == 'Mike Gallagher', 'party'] = 'republican'

In [251]:
df1.loc[df1["candidate"] == 'Frederica S. Wilson', 'candidatevotes'] = 1
df1.loc[df1["candidate"] == 'Frederica S. Wilson', 'totalvotes'] = 1

In [216]:
df1.party.value_counts()

republican                                    11991
democrat                                      10024
independent                                    1104
natural law                                     371
working families                                245
independence                                    229
right to life                                   225
reform                                          173
peace and freedom                               165
constitution                                    140
american independent                            104
american                                         72
none                                             67
no party affiliation                             63
u.s. taxpayers                                   63
independent american                             47
u.s. labor                                       36
women's equality                                 32
other                                            26
citizens    

In [218]:
df1.columns

Index(['year', 'state', 'stage', 'runoff', 'special', 'candidate', 'party',
       'writein', 'candidatevotes', 'totalvotes', 'district_id',
       'yr_district_id'],
      dtype='object')

In [221]:
dem_vote_df = df1[df1.party == 'democrat']

In [381]:
def func(x):
     return pd.Series(dict(total_dem_votes = x['candidatevotes'].sum()))

In [382]:
dem_vote = pd.DataFrame(dem_vote_df.groupby(['yr_district_id']).apply(func))

In [383]:
dem_vote.reset_index(inplace=True)

In [384]:
len(dem_vote)

9079

In [385]:
dem_vote.head()

,yr_district_id,total_dem_votes
0,1976-AK-00,34194
1,1976-AL-01,58906
2,1976-AL-02,66288
3,1976-AL-03,106935
4,1976-AL-04,141490


In [232]:
winner_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806


In [386]:
wd_df = pd.merge(winner_df, dem_vote, on=['yr_district_id'], how='left')
wd_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,total_dem_votes
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938,113553.0
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786,69384.0
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955,110496.0
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563,116217.0
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806,144780.0


In [387]:
rep_vote_df = df1[df1.party == 'republican']

In [388]:
len(rep_vote_df)

11995

In [390]:
def rfunc(x):
     return pd.Series(dict(total_rep_votes = x['candidatevotes'].sum()))

In [391]:
rep_vote = pd.DataFrame(rep_vote_df.groupby(['yr_district_id']).apply(rfunc))

In [392]:
rep_vote.reset_index(inplace=True)

In [393]:
len(rep_vote)

8931

In [394]:
rep_vote.head()

,yr_district_id,total_rep_votes
0,1976-AK-00,83722
1,1976-AL-01,98257
2,1976-AL-02,90069
3,1976-AL-04,34531
4,1976-AL-06,92113


In [395]:
votes_df = pd.merge(wd_df, rep_vote, on=['yr_district_id'], how='left')
votes_df

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,total_dem_votes,total_rep_votes
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0,83722.0
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0,98257.0
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0,90069.0
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0,NaN
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0,34531.0
5,1976-AL-05,113553,113560,1976,Alabama,Ronnie G. Flippo,democrat,False,AL-05,0.999938,113553.0,NaN
6,1976-AL-06,92113,162518,1976,Alabama,"John H. Buchanan, Jr.",republican,False,AL-06,0.566786,69384.0,92113.0
7,1976-AL-07,110496,110501,1976,Alabama,Walter Flowers,democrat,False,AL-07,0.999955,110496.0,NaN
8,1976-AR-01,116217,168782,1976,Arkansas,Bill Alexander,democrat,False,AR-01,0.688563,116217.0,52565.0
9,1976-AR-02,144780,167607,1976,Arkansas,Jim Guy Tucker,democrat,False,AR-02,0.863806,144780.0,22819.0


In [398]:
votes_df['total_dem_vote_share'] = votes_df.total_dem_votes/ votes_df.totalvotes

In [399]:
votes_df['total_rep_vote_share'] = votes_df.total_rep_votes/ votes_df.totalvotes

In [400]:
votes_df.head()

,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,winner_voteshare,total_dem_votes,total_rep_votes,total_dem_vote_share,total_rep_vote_share
0,1976-AK-00,83722,118208,1976,Alaska,Don Young,republican,False,AK-00,0.708260,34194.0,83722.0,0.289270,0.708260
1,1976-AL-01,98257,157170,1976,Alabama,Jack Edwards,republican,False,AL-01,0.625164,58906.0,98257.0,0.374792,0.625164
2,1976-AL-02,90069,156362,1976,Alabama,"William L. """"Bill"""" Dickinson",republican,False,AL-02,0.576029,66288.0,90069.0,0.423939,0.576029
3,1976-AL-03,106935,108048,1976,Alabama,Bill Nichols,democrat,False,AL-03,0.989699,106935.0,NaN,0.989699,NaN
4,1976-AL-04,141490,176022,1976,Alabama,Tom Bevill,democrat,False,AL-04,0.803820,141490.0,34531.0,0.803820,0.196174


In [264]:
votes_df.fillna(0, inplace=True)

In [266]:
votes_df.to_csv('HouseVotesbyDistrict_76_18.csv')

## Feature Engineering

In [290]:
votes_df = pd.read_csv('HouseVotesbyDistrict_76_18.csv')

In [291]:
votes_df['winner'] = votes_df['winner'].map(lambda x: re.sub(r'\W+', '', x))

In [293]:
votes_df.sort_values(['district_id', 'year'], inplace=True)

In [294]:
votes_df['L1_winner']= votes_df.winner.shift()

In [295]:
votes_df['L2_winner']= votes_df.winner.shift(2)

In [296]:
votes_df['L3_winner']= votes_df.winner.shift(3)

In [297]:
votes_df['L4_winner']= votes_df.winner.shift(4)

In [298]:
votes_df['L5_winner']= votes_df.winner.shift(5)

In [306]:
votes_df['L6_winner']= votes_df.winner.shift(6)

In [307]:
votes_df['L7_winner']= votes_df.winner.shift(7)

In [308]:
votes_df['L8_winner']= votes_df.winner.shift(8)

In [309]:
votes_df['L9_winner']= votes_df.winner.shift(9)

In [310]:
votes_df['L10_winner']= votes_df.winner.shift(10)

In [303]:
votes_df['dL1_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L1_winner'], 'dL1_winner']= 1

In [314]:
votes_df['incumbent_party'] = votes_df.winner_party.shift()

In [316]:
votes_df['dL2_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L2_winner'], 'dL2_winner']= 1

In [317]:
votes_df['dL3_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L3_winner'], 'dL3_winner']= 1

In [318]:
votes_df['dL4_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L4_winner'], 'dL4_winner']= 1

In [319]:
votes_df['dL5_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L5_winner'], 'dL5_winner']= 1

In [320]:
votes_df['dL6_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L6_winner'], 'dL6_winner']= 1

In [321]:
votes_df['dL7_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L7_winner'], 'dL7_winner']= 1

In [322]:
votes_df['dL8_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L8_winner'], 'dL8_winner']= 1

In [323]:
votes_df['dL9_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L9_winner'], 'dL9_winner']= 1

In [324]:
votes_df['dL10_winner'] = 0
votes_df.loc[votes_df['winner'] == votes_df['L10_winner'], 'dL10_winner']= 1

In [326]:
votes_df['incumbent_L5_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] + votes_df['dL5_winner']

In [327]:
votes_df['incumbent_L4_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] 

In [328]:
votes_df['incumbent_L3_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] 

In [329]:
votes_df['incumbent_L2_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner']

In [331]:
votes_df['incumbent_L10_races'] = votes_df['dL1_winner'] + votes_df['dL2_winner'] + \
    votes_df['dL3_winner'] + votes_df['dL4_winner'] + votes_df['dL5_winner'] + \
    votes_df['dL6_winner'] + votes_df['dL7_winner'] + votes_df['dL8_winner'] + \
    votes_df['dL9_winner'] + votes_df['dL10_winner']

In [333]:
votes_df['rep_wins'] = 0
votes_df.loc[votes_df['winner_party'] == 'republican', 'rep_wins'] = 1 

In [335]:
votes_df['dem_wins'] = 0
votes_df.loc[votes_df['winner_party'] == 'democrat', 'dem_wins'] = 1 

In [336]:
votes_df['rep_L5_wins'] = votes_df.rep_wins.rolling(window=5).sum()

In [342]:
votes_df['rep_L4_wins'] = votes_df.rep_wins.rolling(window=4).sum()

In [343]:
votes_df['rep_L3_wins'] = votes_df.rep_wins.rolling(window=3).sum()

In [344]:
votes_df['rep_L2_wins'] = votes_df.rep_wins.rolling(window=2).sum()

In [347]:
votes_df['rep_L1_wins'] = 0
votes_df.loc[votes_df['incumbent_party'] == 'republican', 'rep_L1_wins']= 1

In [338]:
votes_df['rep_L10_wins'] = votes_df.rep_wins.rolling(window=10).sum()

In [339]:
votes_df['dem_L10_wins'] = votes_df.dem_wins.rolling(window=10).sum()

In [340]:
votes_df['dem_L5_wins'] = votes_df.dem_wins.rolling(window=5).sum()

In [349]:
votes_df['dem_L4_wins'] = votes_df.dem_wins.rolling(window=4).sum()

In [350]:
votes_df['dem_L3_wins'] = votes_df.dem_wins.rolling(window=3).sum()

In [351]:
votes_df['dem_L2_wins'] = votes_df.dem_wins.rolling(window=2).sum()

In [352]:
votes_df['dem_L1_wins'] = 0
votes_df.loc[votes_df['incumbent_party'] == 'democrat', 'dem_L1_wins']= 1

In [355]:
votes_df['rep_L10_voteshare'] = votes_df.rep_voteshare.rolling(window=10).mean()

In [353]:
votes_df['rep_L5_voteshare'] = votes_df.rep_voteshare.rolling(window=5).mean()

In [356]:
votes_df['rep_L4_voteshare'] = votes_df.rep_voteshare.rolling(window=4).mean()

In [357]:
votes_df['rep_L3_voteshare'] = votes_df.rep_voteshare.rolling(window=3).mean()

In [361]:
votes_df['rep_L2_voteshare'] = votes_df.rep_voteshare.rolling(window=2).mean()

In [362]:
votes_df['rep_L1_voteshare']= votes_df.rep_voteshare.shift()

In [363]:
votes_df['dem_L10_voteshare'] = votes_df.dem_voteshare.rolling(window=10).mean()

In [364]:
votes_df['dem_L5_voteshare'] = votes_df.dem_voteshare.rolling(window=5).mean()

In [365]:
votes_df['dem_L4_voteshare'] = votes_df.dem_voteshare.rolling(window=4).mean()

In [366]:
votes_df['dem_L3_voteshare'] = votes_df.dem_voteshare.rolling(window=3).mean()

In [367]:
votes_df['dem_L2_voteshare'] = votes_df.dem_voteshare.rolling(window=2).mean()

In [368]:
votes_df['dem_L1_voteshare']= votes_df.dem_voteshare.shift()

In [370]:
votes_df['target'] = 0
votes_df.loc[votes_df['winner_party'] == 'democrat', 'target']= 1

In [372]:
votes_df_2012_2018 = votes_df[votes_df.year > 2010]
votes_df_2012_2018

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,rep_L3_voteshare,rep_L2_voteshare,rep_L1_voteshare,dem_L10_voteshare,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target
18,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.627413,0.690409,0.689579,0.337937,0.342113,0.362218,0.347010,0.295641,0.305133,0
19,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.655527,0.638501,0.691240,0.335949,0.371708,0.362675,0.333651,0.347910,0.286149,0
20,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.627765,0.596028,0.585763,0.333620,0.362184,0.340293,0.352013,0.384946,0.409672,0
21,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.574291,0.568555,0.606292,0.342878,0.365229,0.380253,0.411621,0.412595,0.360220,0
40,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.929036,0.902201,0.825839,0.172897,0.137269,0.079549,0.000000,0.000000,0.000000,0
41,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.828657,0.830066,0.978562,0.171946,0.127065,0.079283,0.105710,0.158565,0.000000,0
42,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.874652,0.822697,0.681569,0.138127,0.063426,0.079283,0.105710,0.158565,0.317130,0
43,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.758986,0.797694,0.963825,0.174903,0.136979,0.171224,0.228298,0.183882,0.000000,0
62,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.547275,0.572859,0.509729,0.345410,0.388443,0.414407,0.451100,0.425483,0.487906,0
63,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.606381,0.654707,0.635989,0.351562,0.396654,0.419735,0.392202,0.344350,0.363060,0


In [375]:
votes_df_2012_2018.target.value_counts()

0    918
1    822
Name: target, dtype: int64

In [405]:
dem_vote_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10024 entries, 0 to 29632
Data columns (total 12 columns):
year              10024 non-null object
state             10024 non-null object
stage             10024 non-null object
runoff            7168 non-null object
special           10024 non-null bool
candidate         10024 non-null object
party             10024 non-null object
writein           10024 non-null bool
candidatevotes    10024 non-null int64
totalvotes        10024 non-null int64
district_id       10024 non-null object
yr_district_id    10024 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 881.0+ KB


In [406]:
dem_vote_df.year = dem_vote_df.year.astype(float).fillna(0.0)


/Users/ptw/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [407]:
dem_vote_1218_df = dem_vote_df[dem_vote_df.year > 2010]
dem_vote_1218_df

,year,state,stage,runoff,special,candidate,party,writein,candidatevotes,totalvotes,district_id,yr_district_id
24055,2012.0,Alabama,gen,NaN,False,Therese Ford,democrat,False,103092,283953,AL-02,2012-AL-02
24059,2012.0,Alabama,gen,NaN,False,John Andrew Harris,democrat,False,98141,273930,AL-03,2012-AL-03
24061,2012.0,Alabama,gen,NaN,False,Daniel H. Boman,democrat,False,69706,269118,AL-04,2012-AL-04
24065,2012.0,Alabama,gen,NaN,False,Charlie L. Holley,democrat,False,101772,291293,AL-05,2012-AL-05
24067,2012.0,Alabama,gen,NaN,False,Penny H. Bailey,democrat,False,88267,308102,AL-06,2012-AL-06
24071,2012.0,Alabama,gen,NaN,False,Terri A. Sewell,democrat,False,232520,306558,AL-07,2012-AL-07
24077,2012.0,Alaska,gen,NaN,False,Sharon M. Cissna,democrat,False,82927,289804,AK-00,2012-AK-00
24079,2012.0,Arizona,gen,NaN,False,Ann Kirkpatrick,democrat,False,122774,251595,AZ-01,2012-AZ-01
24082,2012.0,Arizona,gen,NaN,False,Ron Barber,democrat,False,147338,292279,AZ-02,2012-AZ-02
24084,2012.0,Arizona,gen,NaN,False,Ra_l M. Grijalva,democrat,False,98468,168698,AZ-03,2012-AZ-03


In [408]:
top_dem_vote = pd.DataFrame(dem_vote_1218_df.groupby(['yr_district_id']).candidatevotes.max())

In [409]:
top_dem_vote.reset_index(inplace=True)

In [410]:
len(top_dem_vote)

1654

In [415]:
top_dem_vote.columns

Index(['yr_district_id', 'candidatevotes'], dtype='object')

In [417]:
df.columns

Index(['year', 'state', 'stage', 'runoff', 'special', 'candidate', 'party',
       'writein', 'candidatevotes', 'totalvotes', 'district_id',
       'yr_district_id'],
      dtype='object')

In [420]:
top_dem_df = pd.merge(top_dem_vote, df, on=['yr_district_id', 'candidatevotes'], how='left')
top_dem_df

,yr_district_id,candidatevotes,year,state,stage,runoff,special,candidate,party,writein,totalvotes,district_id
0,2012-AK-00,82927,2012,Alaska,gen,NaN,False,Sharon M. Cissna,democrat,False,289804,AK-00
1,2012-AL-02,103092,2012,Alabama,gen,NaN,False,Therese Ford,democrat,False,283953,AL-02
2,2012-AL-03,98141,2012,Alabama,gen,NaN,False,John Andrew Harris,democrat,False,273930,AL-03
3,2012-AL-04,69706,2012,Alabama,gen,NaN,False,Daniel H. Boman,democrat,False,269118,AL-04
4,2012-AL-05,101772,2012,Alabama,gen,NaN,False,Charlie L. Holley,democrat,False,291293,AL-05
5,2012-AL-06,88267,2012,Alabama,gen,NaN,False,Penny H. Bailey,democrat,False,308102,AL-06
6,2012-AL-07,232520,2012,Alabama,gen,NaN,False,Terri A. Sewell,democrat,False,306558,AL-07
7,2012-AR-01,96601,2012,Arkansas,gen,NaN,False,Scott Ellington,democrat,False,246843,AR-01
8,2012-AR-02,113156,2012,Arkansas,gen,NaN,False,Herb Rule,democrat,False,286598,AR-02
9,2012-AR-03,39318,2012,Arkansas,gen,NaN,False,Rebekah J. Kennedy,green,False,245660,AR-03


In [422]:
top_dem_df.drop(['year', 'state', 'stage', 'runoff', 'special', 'party',
                 'writein', 'totalvotes', 'district_id'], axis=1, inplace=True)

In [424]:
top_dem_df.head()

,yr_district_id,candidatevotes,candidate
0,2012-AK-00,82927,Sharon M. Cissna
1,2012-AL-02,103092,Therese Ford
2,2012-AL-03,98141,John Andrew Harris
3,2012-AL-04,69706,Daniel H. Boman
4,2012-AL-05,101772,Charlie L. Holley


In [425]:
top_dem_df.columns = ['yr_district_id', 'top_dem_votes', 'top_dem_candidate']

In [426]:
votes_df_2012_2018_d = pd.merge(votes_df_2012_2018, top_dem_df, on=['yr_district_id'], how='left')
votes_df_2012_2018_d

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,rep_L1_voteshare,dem_L10_voteshare,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target,top_dem_votes,top_dem_candidate
0,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.689579,0.337937,0.342113,0.362218,0.347010,0.295641,0.305133,0,82927.0,Sharon M. Cissna
1,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.691240,0.335949,0.371708,0.362675,0.333651,0.347910,0.286149,0,114602.0,Forrest Dunbar
2,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.585763,0.333620,0.362184,0.340293,0.352013,0.384946,0.409672,0,111019.0,Steve Lindbeck
3,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.606292,0.342878,0.365229,0.380253,0.411621,0.412595,0.360220,0,131199.0,Alyse S. Galvin
4,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.825839,0.172897,0.137269,0.079549,0.000000,0.000000,0.000000,0,NaN,NaN
5,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.978562,0.171946,0.127065,0.079283,0.105710,0.158565,0.000000,0,48278.0,Burton R. LeFlore
6,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.681569,0.138127,0.063426,0.079283,0.105710,0.158565,0.317130,0,NaN,NaN
7,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.963825,0.174903,0.136979,0.171224,0.228298,0.183882,0.000000,0,89226.0,Robert Kennedy Jr.
8,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.509729,0.345410,0.388443,0.414407,0.451100,0.425483,0.487906,0,103092.0,Therese Ford
9,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.635989,0.351562,0.396654,0.419735,0.392202,0.344350,0.363060,0,54692.0,Erick Wright


In [427]:
rep_vote_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11995 entries, 1 to 29633
Data columns (total 12 columns):
year              11995 non-null object
state             11995 non-null object
stage             11995 non-null object
runoff            8563 non-null object
special           11995 non-null bool
candidate         11995 non-null object
party             11995 non-null object
writein           11995 non-null bool
candidatevotes    11995 non-null int64
totalvotes        11995 non-null int64
district_id       11995 non-null object
yr_district_id    11995 non-null object
dtypes: bool(2), int64(2), object(8)
memory usage: 1.0+ MB


In [428]:
rep_vote_df.year = rep_vote_df.year.astype(float).fillna(0.0)


In [429]:
rep_vote_1218_df = rep_vote_df[rep_vote_df.year > 2010]
rep_vote_1218_df

,year,state,stage,runoff,special,candidate,party,writein,candidatevotes,totalvotes,district_id,yr_district_id
24054,2012.0,Alabama,gen,NaN,False,Jo Bonner,republican,False,196374,200676,AL-01,2012-AL-01
24057,2012.0,Alabama,gen,NaN,False,Martha Roby,republican,False,180591,283953,AL-02,2012-AL-02
24058,2012.0,Alabama,gen,NaN,False,Mike Rogers,republican,False,175306,273930,AL-03,2012-AL-03
24063,2012.0,Alabama,gen,NaN,False,Robert B. Aderholt,republican,False,199071,269118,AL-04,2012-AL-04
24066,2012.0,Alabama,gen,NaN,False,Mo Brooks,republican,False,189185,291293,AL-05,2012-AL-05
24069,2012.0,Alabama,gen,NaN,False,Spencer Bachus,republican,False,219262,308102,AL-06,2012-AL-06
24070,2012.0,Alabama,gen,NaN,False,Don Chamberlain,republican,False,73835,306558,AL-07,2012-AL-07
24073,2012.0,Alaska,gen,NaN,False,Jim C. McDermott,republican,False,15028,289804,AK-00,2012-AK-00
24075,2012.0,Alaska,gen,NaN,False,Don Young,republican,False,185296,289804,AK-00,2012-AK-00
24078,2012.0,Arizona,gen,NaN,False,Kim Allen,republican,False,15227,251595,AZ-01,2012-AZ-01


In [430]:
top_rep_vote = pd.DataFrame(rep_vote_1218_df.groupby(['yr_district_id']).candidatevotes.max())

In [431]:
top_rep_vote.reset_index(inplace=True)

In [432]:
len(top_rep_vote)

1637

In [433]:
top_rep_vote.columns

Index(['yr_district_id', 'candidatevotes'], dtype='object')

In [417]:
df.columns

Index(['year', 'state', 'stage', 'runoff', 'special', 'candidate', 'party',
       'writein', 'candidatevotes', 'totalvotes', 'district_id',
       'yr_district_id'],
      dtype='object')

In [434]:
top_rep_df = pd.merge(top_rep_vote, df, on=['yr_district_id', 'candidatevotes'], how='left')
top_rep_df

,yr_district_id,candidatevotes,year,state,stage,runoff,special,candidate,party,writein,totalvotes,district_id
0,2012-AK-00,185296,2012,Alaska,gen,NaN,False,Don Young,republican,False,289804,AK-00
1,2012-AL-01,196374,2012,Alabama,gen,NaN,False,Jo Bonner,republican,False,200676,AL-01
2,2012-AL-02,180591,2012,Alabama,gen,NaN,False,Martha Roby,republican,False,283953,AL-02
3,2012-AL-03,175306,2012,Alabama,gen,NaN,False,Mike Rogers,republican,False,273930,AL-03
4,2012-AL-04,199071,2012,Alabama,gen,NaN,False,Robert B. Aderholt,republican,False,269118,AL-04
5,2012-AL-05,189185,2012,Alabama,gen,NaN,False,Mo Brooks,republican,False,291293,AL-05
6,2012-AL-06,219262,2012,Alabama,gen,NaN,False,Spencer Bachus,republican,False,308102,AL-06
7,2012-AL-07,73835,2012,Alabama,gen,NaN,False,Don Chamberlain,republican,False,306558,AL-07
8,2012-AR-01,138800,2012,Arkansas,gen,NaN,False,"Eric A. """"Rick"""" Crawford",republican,False,246843,AR-01
9,2012-AR-02,158175,2012,Arkansas,gen,NaN,False,Tim Griffin,republican,False,286598,AR-02


In [435]:
top_rep_df.drop(['year', 'state', 'stage', 'runoff', 'special', 'party',
                 'writein', 'totalvotes', 'district_id'], axis=1, inplace=True)

In [436]:
top_rep_df.head()

,yr_district_id,candidatevotes,candidate
0,2012-AK-00,185296,Don Young
1,2012-AL-01,196374,Jo Bonner
2,2012-AL-02,180591,Martha Roby
3,2012-AL-03,175306,Mike Rogers
4,2012-AL-04,199071,Robert B. Aderholt


In [437]:
top_rep_df.columns = ['yr_district_id', 'top_rep_votes', 'top_rep_candidate']

In [438]:
votes_df_2012_2018_dr = pd.merge(votes_df_2012_2018_d, top_rep_df, on=['yr_district_id'], how='left')
votes_df_2012_2018_dr

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target,top_dem_votes,top_dem_candidate,top_rep_votes,top_rep_candidate
0,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.342113,0.362218,0.347010,0.295641,0.305133,0,82927.0,Sharon M. Cissna,185296.0,Don Young
1,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.371708,0.362675,0.333651,0.347910,0.286149,0,114602.0,Forrest Dunbar,142572.0,Don Young
2,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.362184,0.340293,0.352013,0.384946,0.409672,0,111019.0,Steve Lindbeck,155088.0,Don Young
3,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.365229,0.380253,0.411621,0.412595,0.360220,0,131199.0,Alyse S. Galvin,149779.0,Don Young
4,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.137269,0.079549,0.000000,0.000000,0.000000,0,NaN,NaN,196374.0,Jo Bonner
5,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.127065,0.079283,0.105710,0.158565,0.000000,0,48278.0,Burton R. LeFlore,103758.0,Bradley Byrne
6,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.063426,0.079283,0.105710,0.158565,0.317130,0,NaN,NaN,208083.0,Bradley Byrne
7,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.136979,0.171224,0.228298,0.183882,0.000000,0,89226.0,Robert Kennedy Jr.,153228.0,Bradley Byrne
8,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.388443,0.414407,0.451100,0.425483,0.487906,0,103092.0,Therese Ford,180591.0,Martha Roby
9,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.396654,0.419735,0.392202,0.344350,0.363060,0,54692.0,Erick Wright,113103.0,Martha Roby


In [446]:
votes_df_2012_2018_dr['top_dem_candidate'].fillna('None', inplace=True)
votes_df_2012_2018_dr['top_rep_candidate'].fillna('None', inplace=True)

In [447]:
votes_df_2012_2018_dr['top_dem_candidate'] = votes_df_2012_2018_dr['top_dem_candidate'].map(lambda x: re.sub(r'\W+', '', x))
votes_df_2012_2018_dr['top_rep_candidate'] = votes_df_2012_2018_dr['top_rep_candidate'].map(lambda x: re.sub(r'\W+', '', x))


In [448]:
votes_df_2012_2018_dr

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,dem_L5_voteshare,dem_L4_voteshare,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target,top_dem_votes,top_dem_candidate,top_rep_votes,top_rep_candidate
0,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.342113,0.362218,0.347010,0.295641,0.305133,0,82927.0,SharonMCissna,185296.0,DonYoung
1,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.371708,0.362675,0.333651,0.347910,0.286149,0,114602.0,ForrestDunbar,142572.0,DonYoung
2,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.362184,0.340293,0.352013,0.384946,0.409672,0,111019.0,SteveLindbeck,155088.0,DonYoung
3,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.365229,0.380253,0.411621,0.412595,0.360220,0,131199.0,AlyseSGalvin,149779.0,DonYoung
4,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.137269,0.079549,0.000000,0.000000,0.000000,0,NaN,None,196374.0,JoBonner
5,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.127065,0.079283,0.105710,0.158565,0.000000,0,48278.0,BurtonRLeFlore,103758.0,BradleyByrne
6,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.063426,0.079283,0.105710,0.158565,0.317130,0,NaN,None,208083.0,BradleyByrne
7,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.136979,0.171224,0.228298,0.183882,0.000000,0,89226.0,RobertKennedyJr,153228.0,BradleyByrne
8,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.388443,0.414407,0.451100,0.425483,0.487906,0,103092.0,ThereseFord,180591.0,MarthaRoby
9,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.396654,0.419735,0.392202,0.344350,0.363060,0,54692.0,ErickWright,113103.0,MarthaRoby


In [450]:
votes_df_2012_2018_dr.to_csv('HouseVotesFeatures_12_18.csv', index= False)

In [453]:
votes_df_2012_2018_dr = pd.read_csv('HouseVotesFeatures_12_18.csv')

In [457]:
votes_df_2012_2018_dr

,Unnamed: 0,yr_district_id,winner_votes,totalvotes,year,state,winner,winner_party,writein,district_id,...,dem_L3_voteshare,dem_L2_voteshare,dem_L1_voteshare,target,top_dem_votes,top_dem_candidate,top_rep_votes,top_rep_candidate,dem_incumbent_in_race,rep_incumbent_in_race
0,7813,2012-AK-00,185296,289804,2012,Alaska,DonYoung,republican,False,AK-00,...,0.347010,0.295641,0.305133,0,82927.0,SharonMCissna,185296.0,DonYoung,0,1
1,8248,2014-AK-00,142572,279741,2014,Alaska,DonYoung,republican,False,AK-00,...,0.333651,0.347910,0.286149,0,114602.0,ForrestDunbar,142572.0,DonYoung,0,1
2,8683,2016-AK-00,155088,308198,2016,Alaska,DonYoung,republican,False,AK-00,...,0.352013,0.384946,0.409672,0,111019.0,SteveLindbeck,155088.0,DonYoung,0,1
3,9118,2018-AK-00,149779,282166,2018,Alaska,DonYoung,republican,False,AK-00,...,0.411621,0.412595,0.360220,0,131199.0,AlyseSGalvin,149779.0,DonYoung,0,1
4,7814,2012-AL-01,196374,200676,2012,Alabama,JoBonner,republican,False,AL-01,...,0.000000,0.000000,0.000000,0,NaN,None,196374.0,JoBonner,0,1
5,8249,2014-AL-01,103758,152234,2014,Alabama,BradleyByrne,republican,False,AL-01,...,0.105710,0.158565,0.000000,0,48278.0,BurtonRLeFlore,103758.0,BradleyByrne,0,0
6,8684,2016-AL-01,208083,215893,2016,Alabama,BradleyByrne,republican,False,AL-01,...,0.105710,0.158565,0.317130,0,NaN,None,208083.0,BradleyByrne,0,1
7,9119,2018-AL-01,153228,242617,2018,Alabama,BradleyByrne,republican,False,AL-01,...,0.228298,0.183882,0.000000,0,89226.0,RobertKennedyJr,153228.0,BradleyByrne,0,1
8,7815,2012-AL-02,180591,283953,2012,Alabama,MarthaRoby,republican,False,AL-02,...,0.451100,0.425483,0.487906,0,103092.0,ThereseFord,180591.0,MarthaRoby,0,1
9,8250,2014-AL-02,113103,167952,2014,Alabama,MarthaRoby,republican,False,AL-02,...,0.392202,0.344350,0.363060,0,54692.0,ErickWright,113103.0,MarthaRoby,0,1


In [455]:
votes_df_2012_2018_dr['dem_incumbent_in_race'] = 0
votes_df_2012_2018_dr.loc[votes_df_2012_2018_dr['top_dem_candidate']
                          == votes_df_2012_2018_dr['L1_winner'], 'dem_incumbent_in_race'] = 1

In [456]:
votes_df_2012_2018_dr['rep_incumbent_in_race'] = 0
votes_df_2012_2018_dr.loc[votes_df_2012_2018_dr['top_rep_candidate']
                          == votes_df_2012_2018_dr['L1_winner'], 'rep_incumbent_in_race'] = 1

In [458]:
votes_df_2012_2018_dr.to_csv('HouseVotesFeatures_12_18.csv', index= False)

## Notes

In [70]:
df[df.party == 'democrat'].groupby(['district_id','year']).candidatevotes.sum()

district_id  year
AK-00        1976     34194
             1978     55176
             1980     39922
             1982     52011
             1984     86052
             1986     74053
             1988     71881
             1990     91677
             1992    102378
             1994     68172
             1996     85114
             1998     77232
             2000     45372
             2002     39357
             2004     67074
             2006     93879
             2008    142560
             2010     77606
             2012     82927
             2014    114602
             2016    111019
             2018    131199
AL-01        1976     58906
             1978     40450
             1982     54315
             1984     98455
             1988     77670
             1992     78742
             1994     50227
             1996     69470
                      ...  
WV-04        1976     73626
             1978     70035
             1980    117595
             1982     91184
  

In [100]:
def func(x):
     return pd.Series(dict(year = x['year'].mean(),
                           state = x['state'].unique()[0], 
                           special = x['special'].mean(), 
                           party = x['party'].unique()[0],
                           writein = x['writein'].mean(),
                           candidatevotes = x['candidatevotes'].max(), 
                           totalvotes = x['totalvotes'].mean(),
                           district_id = x['district_id'].unique()[0],
                           voteshare = x['voteshare'].max(),
                           yr_district_id = x['yr_district_id'].unique()[0]))

In [49]:
fc_ga = fc_all.groupby(['recipient_parent_name1']).apply(func)